In [3]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from highcharts import Highchart
#!pip import python_highcharts
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
x_train = np.load("x_train.npy")
x_test = np.load("x_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")
new_x = np.reshape(x_train, (10660, 5700))
new_xtest = np.reshape(x_test,(3554, 5700))

In [ ]:
# set parameters:
batch_size = 128
nb_epoch = 2500
n_RNN_nodes = 1024
recurrent_dropout = 0.25
drop_out = 0.5
model = Sequential()
model.add(Bidirectional(LSTM(units=n_RNN_nodes, return_sequences=True, recurrent_dropout=recurrent_dropout, dropout=drop_out, name='hidden_layer1', kernel_initializer=initializers.random_normal(stddev=0.01))))
model.add(Bidirectional(LSTM(units=n_RNN_nodes, return_sequences=False, recurrent_dropout=recurrent_dropout,dropout=drop_out, name='hidden_layer2', kernel_initializer=initializers.random_normal(stddev=0.01))))
model.add(Dropout(drop_out))
model.add(Dense(units=256, activation='relu', name='dense_layer', kernel_initializer=initializers.random_normal(stddev=0.01)))
model.add(Dropout(drop_out))
model.add(Dense(1, activation='sigmoid', name='output_layer', kernel_initializer=initializers.random_normal(stddev=0.01)))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=10, verbose=0, mode='auto', epsilon=0.0001, min_lr=0.001)
history = model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_split=0.3, shuffle = True, callbacks=[reduce_lr], verbose = 1) 
score = model.evaluate(x_test, y_test)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
fig = plt.figure()#
model_id = 'final.h5'
plt.plot(history.history['accuracy'],label='training acc')
plt.plot(history.history['val_accuracy'],label='val acc')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='lower right')
fig.savefig('VGG16'+str(model_id)+'acc.png')
fig = plt.figure()
plt.plot(history.history['loss'],label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper right')
fig.savefig('VGG16'+str(model_id)+'loss.png')
model.save("final.h5")